<a href="https://colab.research.google.com/github/shivangibithel/ACMR_demo/blob/master/Wiki_ACMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 2.7.17


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# %cd gdrive/
# %cd gdrive/MyDrive/Multimodal-IR-Ablation/JFSE/ACMR_demo/
%tensorflow_version 1.x
%cd gdrive/MyDrive/ACMR_demo
!ls

In [ ]:
!ls

# One liner

In [4]:
# !git clone https://github.com/Prabhat1808/Multimodal-IR-Ablation.git

In [5]:
# !git clone https://github.com/shivangibithel/ACMR_demo.git

In [6]:
# !python train_adv_crossmodal_triplet_wiki.py

In [7]:
# !python train_adv_crossmodal_simple_nuswide.py

In [21]:
# !python train_adv_crossmodal_simple_wiki.py

Streaming output truncated to the last 5000 lines.
Epoch: [48][  20/  33] time: 36.7801, emb_loss: 12.91121960, domain_loss: 1.36712468, label_loss: 0.08058617, similar_loss: 4.79969883, dissimilar_loss: 2.64520931
Epoch: [48][  21/  33] time: 36.7891, emb_loss: 12.42904854, domain_loss: 1.38603675, label_loss: 0.07713366, similar_loss: 4.65960550, dissimilar_loss: 2.80390072
Epoch: [48][  22/  33] time: 36.7981, emb_loss: 13.26291656, domain_loss: 1.38540244, label_loss: 0.08262999, similar_loss: 4.96483421, dissimilar_loss: 1.75413799
Epoch: [48][  23/  33] time: 36.8071, emb_loss: 12.49704552, domain_loss: 1.38143289, label_loss: 0.07989389, similar_loss: 4.45696878, dissimilar_loss: 2.53438210
Epoch: [48][  24/  33] time: 36.8248, emb_loss: 12.48396397, domain_loss: 1.36983573, label_loss: 0.07764953, similar_loss: 4.68241501, dissimilar_loss: 1.82975721
Epoch: [48][  25/  33] time: 36.8339, emb_loss: 13.06384659, domain_loss: 1.38056731, label_loss: 0.08175580, similar_loss: 4.834

# Framework

In [9]:
import os
import os.path as path
import numpy as np
import random
from sys import getsizeof

"""
    Dataset is the superclass which contains high level variables:
    x_{train,val,test}, y_{train,val,test}
    dir_{train,val,test}
    loader : function to read the dataset
    preprocess : function to preprocess the datasets
    normalize : normalize train, val and test datasets
    Note: while normalizing:
        1. stats obtained from train can be used to normalize val and test
        2. val and test can be normalized independently
        This needs to be fixed.... #LOOK
    Example code is provided for NUS-wide
"""
class Dataset:

    # directories -> tuple of training, val, test
    # each of directory -> x_train.txt, y_train.txt and any other files
    # x_train -> {image: ...
    #               text: ....
    #                   embeddings: ...}
    def __init__(self, directories,
                         loader, 
                         preprocess=None,
                         preprocess_params=None,
                         normalize=None,
                         normalization_params=None,
                         read_directories=(True,True,True),
                         summarize=None
                         ):
        self.dir_train, self.dir_val, self.dir_test = directories
        self.read_train, self.read_val, self.read_test = read_directories
        self.x_train = None
        self.x_val = None
        self.x_test = None
        self.y_train = None
        self.y_val = None
        self.y_test = None
        self.loader = loader
        self.preprocess = preprocess
        self.preprocess_params = preprocess_params
        self.normalize = normalize
        self.normalization_params = normalization_params
        self.summarize = summarize
        self.stats = None

    def get_train_labels(self):
        return self.y_train

    def get_test_labels(self):
        return self.y_test

    def check_directory(self, dir, msg='{} Directory does not exist!!'):
        assert path.exists(dir), msg.format(dir)

    def load_data(self):
        if self.read_train:
            self.check_directory(self.dir_train)
            self.x_train, self.y_train = self.loader(self.dir_train, "train") #TODO changed
        if self.read_val:
            self.check_directory(self.dir_val)
            self.x_val, self.y_val = self.loader(self.dir_val, "val") #TODO changed
        if self.read_test:
            self.check_directory(self.dir_test)
            self.x_test, self.y_test = self.loader(self.dir_test, "test") #TODO changed

    def preprocess_data(self):
        if not self.preprocess:
            def func(x,y, params=None):
                return x,y
            self.preprocess = func

        if not self.normalize:
            def func(x,y, params=None):
                return x,y
            self.normalize = func

        if self.read_train:
            self.x_train, self.y_train = self.preprocess(self.x_train, self.y_train, self.preprocess_params) #TODO tag here
            self.x_train, self.y_train = self.normalize(self.x_train, self.y_train, self.normalization_params)
        if self.read_val:
            self.x_val, self.y_val = self.preprocess(self.x_val, self.y_val, self.preprocess_params)
            self.x_val, self.y_val = self.preprocess(self.x_val, self.y_val, self.normalization_params)
        if self.read_test:
            self.x_test, self.y_test = self.preprocess(self.x_test, self.y_test, self.preprocess_params)
            self.x_test, self.y_test = self.preprocess(self.x_test, self.y_test, self.normalization_params)

    def get_stats(self):
        #TODO: following 2 lines added
        if (self.summarize == None):
            return None

        if self.stats:
            return self.stats

        stats = {}
        if self.read_train:
            stats['train'] = self.summarize(self.x_train, self.y_train)
        if self.read_test:
            stats['test'] = self.summarize(self.x_test, self.y_test)
        if self.read_val:
            stats['val'] = self.summarize(self.x_val, self.y_val)

        self.stats = stats
        return self.stats

In [10]:
import os
import os.path as path
import numpy as np
import random
import sys
import time
import json


class Parameters:
    def __init__(self, values):
        self.values = values
        # This should ideally be stored in a dictionary format,
        # to allow for easy access and referencing
        self.size = sys.getsizeof(values)


def map_rank(traingnd, testgnd, hamming_rank):
    """ 
        This funtion returns map@all metric score.
        hamming_rank : numtrain x numtest
        *gnd : numsamples x labelsize
    """
    numtrain, numtest = hamming_rank.shape
    apall = np.zeros((numtrain, numtest))
    patk = np.zeros((numtrain, numtest))
    ratk = np.zeros((numtrain, numtest))
    aa = np.array([i+1 for i in range(numtrain)])
    for i in range(numtest):
        y = hamming_rank[:, i]
        new_label = np.array([0 for j in range(numtrain)])
        relevant_indices = (np.matmul(traingnd, testgnd[i, :].reshape((-1, 1))) > 0).reshape(-1)
        new_label[relevant_indices] = 1
        total_relevant = np.sum(new_label)
        xx = np.cumsum(new_label[y]) #retrieved relevant
        ratk[:, i] = xx / total_relevant
        patk[:, i] = xx / aa
        x = xx * new_label[y]
        p = x / aa #precision@k
        p = np.cumsum(p)
        mask = (p != 0)
        p[mask] = p[mask]/xx[mask]
        apall[:, i] = p.copy()
    pre = np.mean(patk, axis=1)
    recall = np.mean(ratk, axis=1)
    mAP = np.mean(apall, axis=1)
    return mAP, pre, recall

class Model:
    # Note: if you are using a neural model, then:
    #   1. pass is_neural = True
    #   2. params can be the saved object of the neural model
    # This is to avoid redundancy, as TF, PyTorch, etc already have optimized saving and loading routines
    def __init__(self, 
                    training_function,
                    hyperparams,
                    dataset_obj,
                    params=None,
                    params_verification=None,
                    prediction_function=None,
                    evaluation_metrics=None,
                    is_neural=None
                    ):
        # training_function -> function that acts on the input data and returns a Parameters object
        # The subsequent inputs should be consistent with the inputs required by the training function
        # hyperparams -> Dictionary of iterations, learning rate or any other vars.
        # params -> an object of class Parameters, which contains the matrices, weights and any other parameters,
        #           in the correct shape and data type, as per the training function
        # params_verification -> ensures consistency b/w params and output of training function
        # prediction_function -> takes params and datapoint(s) as input to generate output(s)
        # evaluation_metrics -> list of evaluation metrics to be calculated
        #                       Output from prediction function should be in the required format
        #                       Implementation for those not provided by the framework
        self.train = training_function
        self.hyperparams = hyperparams
        # dataset_object has to be of type Dataset
        self.dataset_obj = dataset_obj
        self.params = params
        self.params_verification = params_verification
        self.prediction_function = prediction_function
        self.evaluation_metrics = evaluation_metrics
        self.is_neural = is_neural
        self.stats = self.initialize_stats()
        self.logs = []
        self.results ={}
        

    def initialize_stats(self):
        stats = {
            'data_stats' : self.dataset_obj.get_stats(),
            'params_size' : self.params.size if self.is_neural else sys.getsizeof(self.params),
            'epochs' : 0,
            'training_time' : 0,
            'loss_history' : {},
            # Dictionary, where multiple lists are stored, for each kind of loss being monitored
            'metrics' : {}
        }

        return stats

    def train_model(self):
        # returns model stats as well
        # note starting and ending time before training_function() call
        # keep track of hyperparams
        # keep track of the size Params object and the number of params therein
        
        start = time.time()
        params, losses, logs = self.train(
                                    self.dataset_obj,
                                    self.params,
                                    self.hyperparams
                                )
        end = time.time()

        self.params = params
        self.stats['loss_history'] = losses
        self.logs.append(logs)
        self.stats['training_time'] = (end-start)

    # tag = train, val, test. Or anything else, but then custom dataset and provisions in prediction_function are needed
    def predict(self, tag):
        """ calls user-defined prediction_function
            user must return the results in the following form"
            type(results) = dict
            results.keys() : ['itot_ranked_results', 'ttoi_ranked_results']
            and matrices corresponding to the each key.
        """
        start = time.time()
        n_samples, results, logs = self.prediction_function(self.dataset_obj, self.params, tag)
        end = time.time()

        self.results[tag] = results #TODO: ''.format(tag)
        self.stats['prediction_time'] = (end-start)/n_samples
        self.logs.append(logs)

    def evaluate(self, train_labels, test_labels):
        """ calculate mAP, recall, pecision
            This funtion will make use of the "results" stored in
            the predict part above.
            args:
                train_labels : #samples x #classes matrix. 
                                It should be a binary matrix saying 
                                given sample belongs what labels.
                test_labels :  #samples x #classes matrix
                                It should be a binary matrix saying 
                                given sample belongs what labels.
        """
        mAP_itot, pre_itot, recall_itot = map_rank(train_labels,test_labels, self.results['test']['itot_ranked_results'].T)
        mAP_ttoi, pre_ttoi, recall_ttoi = map_rank(train_labels, test_labels, self.results['test']['ttoi_ranked_results'].T)
        self.stats['metrics']['map_itot'] = mAP_itot
        self.stats['metrics']['map_ttoi'] = mAP_ttoi
        self.stats['metrics']['pre_itot'] = pre_itot
        self.stats['metrics']['pre_ttoi'] = pre_ttoi
        self.stats['metrics']['recall_itot'] = recall_itot
        self.stats['metrics']['recall_ttoi'] = recall_ttoi
        print('image to text mAP@max: ', np.max(mAP_itot), np.argmax(mAP_itot))
        print('image to text P@max: ', np.max(pre_itot), np.argmax(pre_itot))
        print('image to text recall@p_max: ', recall_itot[np.argmax(pre_itot)])
        print('text to image mAP@max: ', np.max(mAP_ttoi), np.argmax(mAP_ttoi))
        print('text to image P@max: ', np.max(pre_ttoi), np.argmax(pre_ttoi))
        print('text to image recall@p_max: ', recall_ttoi[np.argmax(pre_ttoi)])
        

    def get_stats(self):
        return self.stats

    def save_stats(self, filename):
        np.save(filename, self.stats)
        # out_data = self.stats
        # outfile = open(filename,'w')
        # json.dump(out_data, outfile, indent=4)

In [11]:
import os
import os.path as path
import numpy as np
import random
import os.path as path
import sys
import time
import json
import matplotlib.pyplot as plt

class Comparator:
    def __init__(self, file1, file2):
        self.stats1 = np.load(file1, allow_pickle=True)[()]
        self.stats2 = np.load(file2, allow_pickle=True)[()]
        self.label1 = path.basename(file1).split('.')[0]
        self.label2 = path.basename(file2).split('.')[0]
        self.comparisions = {}
        self.outdir = '{}_V_{}'.format(self.label1, self.label2)
        if not path.exists(self.outdir):
            os.makedirs(self.outdir)

    # creates a bar plot
    # supported tags - params_size, training_time, inference_time
    # other tags can also be used, but consistency has to be checked by the user
    def createBarPlot(self, tag):
        labels = [self.label1, self.label2]
        param_sizes = [self.stats1[tag], self.stats2[tag]]
        plt.title(tag)
        plt.bar(labels, param_sizes)
        outfile = path.join(self.outdir, '{}.jpeg'.format(tag))
        plt.savefig(outfile)
        plt.close()
    
    # supported tags: loss_histry, metrics
    # If the metric object is 2-level, then a subtag can be provided as well
    # For example -> self.stats1[tag][subtag]
    def createLinePlot(self, tag, subtag=''):       
        line1 = self.stats1[tag]
        if subtag != '':
            line1 = line1[subtag]
        indices1 = [i for i in range(1, len(line1)+1)]
        plt.plot(indices1, line1, label=self.label1)
        
        line2 = self.stats2[tag]
        if subtag != '':
            line2 = line2[subtag]
        indices2 = [i for i in range(1, len(line2)+1)]

        plt.title(tag+ ' ' + subtag)
        plt.ylabel(tag + ' ' + subtag)
        plt.xlabel('i')
        plt.plot(indices2, line2, label=self.label2)
        plt.legend()
        outfile = path.join(self.outdir, '{}.jpeg'.format(tag+' '+subtag))
        plt.savefig(outfile)
        plt.close()

# Base Model

In [12]:
import os
import json
import tensorflow as tf

class BaseDataIter(object):
    def __init__(self, batch_size):
        self.batch_size = batch_size

    def train_data(self):
        raise NotImplemented

    def test_data(self):
        raise NotImplemented
    

class BaseModelParams(object):
    """
    Base class for model parameters
    Any model that takes parameters should derive this class to provide parameters
    """
    def __init__(self):
        """
        Common parameters
        Derived classes should override these parameters
        """
        # Checkpoint root directory; it may contain multiple directories for
        # different models
        self.checkpoint_dir = None

        # Sample directory
        self.sample_dir = None

        # Log directory
        self.log_dir = None

        # Dataset directory; this is the root directory of all datasets.
        # E.g., if dataset coco is located at /mnt/data/coco, then this
        # value should be /mnt/data
        self.dataset_dir = None

        # Name of the dataset; it should be the same as the directory
        # name containing this dataset.
        # E.g., if dataset coco is located at /mnt/data/coco, then this
        # value should be coco
        self.dataset_name = None

        # Name of this model; it is used as the base name for checkpoint files
        self.model_name = None

        # Name of the directory containing the checkpoint files.
        # This can be the same as the model name; however, it can also be encoded
        # to contain certain details of a particular model.
        # This directory will be a subdirectory under checkpoint directory.
        self.model_dir = None

        # Checkpoint file to load
        self.ckpt_file = None

    def load(self, f):
        """
        Load parameters from specified json file.
        The loaded parameters override those with the same name defined in this subclasses
        :param f:
        :return:
        """
        self.__dict__ = json.load(f)

    def loads(self, s):
        """
        Load parameters from json string
        The loaded parameters override those with the same name defined in this subclasses
        :param s:
        :return:
        """
        self.__dict__ = json.loads(s)

    def update(self):
        """
        Update the params
        :return:
        """
        raise Exception('Not implemented')


class BaseModel(object):
    """
    Base class for models
    """
    def __init__(self, model_params=None):
        """

        """
        self.model_params = model_params
        # self.dataIter = dataIter
        self.saver = None

    def get_checkpoint_dir(self):
        """
        Get the dir for all checkpoints.
        Implemented by the derived classes.
        :return:
        """
        if self.model_params is not None and self.model_params.checkpoint_dir is not None:
            return self.model_params.checkpoint_dir
        else:
            raise Exception('get_checkpoint_dir must be implemented by derived classes')

    def get_model_dir(self):
        """
        Get the model dir for the checkpoint
        :return:
        """
        if self.model_params is not None and self.model_params.model_dir is not None:
            return self.model_params.model_dir
        else:
            raise Exception('get_model_dir must be implemented by derived classes')

    def get_model_name(self):
        """
        Get the base model name.
        Implemented by the derived classes.
        :return:
        """
        if self.model_params is not None and self.model_params.model_name is not None:
            return self.model_params.model_name
        else:
            raise Exception('get_model_name must be implemented by derived classes')

    def get_sample_dir(self):
        """
        Get the dir for samples.
        Implemented by the derived classes.
        :return:
        """
        if self.model_params is not None and self.model_params.sample_dir is not None:
            return self.model_params.sample_dir
        else:
            raise Exception('get_sample_dir must be implemented by derived classes')

    def get_dataset_dir(self):
        """
        Get the dataset dir.
        Implemented by the derived classes.
        :return:
        """
        if self.model_params is not None and self.model_params.dataset_dir is not None:
            return self.model_params.dataset_dir
        else:
            raise Exception('get_dataset_dir must be implemented by derived classes')

    def check_dirs(self):
        if not os.path.exists(self.get_sample_dir()):
            os.mkdir(self.get_sample_dir())

        # sanity check for dataset
        if not os.path.exists(self.get_dataset_dir()):
            raise Exception('Dataset dir %s does not exist' % self.get_dataset_dir())

    def save(self, step, sess):
        checkpoint_dir = os.path.join(self.get_checkpoint_dir(), self.get_model_dir())

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(sess,
                        os.path.join(checkpoint_dir, self.get_model_name()),
                        global_step=step)

    def load(self, sess):
        """
        Load from a specified directory.
        This is for resuming training from a previous snapshot and is called from train(),
        therefore, a saver is created in train()

        Args:
            sess: tf session
        """
        print(' [*] Reading checkpoints...')

        checkpoint_dir = os.path.join(self.get_checkpoint_dir(), self.get_model_dir())

        ckpt_path = tf.train.latest_checkpoint(checkpoint_dir)
        if ckpt_path is not None:
            self.saver.restore(sess, ckpt_path)
            return True
        else:
            return False

    def load_for_testing(self, ckpt_path, sess):
        """
        Load from specified checkpoint file.
        This is for testing the model, a saver will be created here to restore the variables

        Args:
            ckpt_path: path to the checkpoint file
            sess: tf session
        """
        print(' [*] Reading checkpoints...')

        if not os.path.exists(ckpt_path):
            return False

        self.saver = tf.train.Saver()
        self.saver.restore(sess, ckpt_path)
        return True


In [13]:
import tensorflow as tf
from tensorflow.python.framework import ops


class FlipGradientBuilder(object):
    def __init__(self):
        self.num_calls = 0

    def __call__(self, x, l=1.0):
        grad_name = "FlipGradient%d" % self.num_calls
        @ops.RegisterGradient(grad_name)
        def _flip_gradients(op, grad):
            return [tf.negative(grad) * l]
        
        g = tf.get_default_graph()
        with g.gradient_override_map({"Identity": grad_name}):
            y = tf.identity(x)
            
        self.num_calls += 1
        return y
    
flip_gradient = FlipGradientBuilder()


# Adversarial class


In [14]:
from __future__ import print_function
import os, time, cPickle
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from random import shuffle
import sklearn.preprocessing
# from base_model import BaseModel, BaseModelParams, BaseDataIter
#import utils
# from flip_gradient import flip_gradient

class DataIter(BaseDataIter):
    def __init__(self, batch_size,data):
        BaseDataIter.__init__(self, batch_size)
        data = data
        self.num_train_batch = 0
        self.num_test_batch = 0
        self.train_img_feats=data.x_train['img_train']
        self.train_txt_vecs=data.x_train['txt_train']
        self.train_labels=data.y_train['train_labels']
        self.test_img_feats=data.x_test['img_test']
        self.test_txt_vecs=data.x_test['txt_test']
        self.test_labels=data.y_test['test_labels']
        self.train_ids=data.y_train['train_ids']
        self.test_ids=data.y_test['test_ids']
        self.train_labels_single=data.y_train['train_labels_single']
        self.test_labels_single=data.y_test['test_labels_single']      
                
        np.random.shuffle(self.train_ids)
        # np.random.shuffle(self.test_ids)          
        self.num_train_batch = len(self.train_ids) / self.batch_size
        self.num_test_batch = len(self.test_ids) / self.batch_size


    def train_data(self):
        for i in range(self.num_train_batch):
            batch_img_ids = self.train_ids[i*self.batch_size : (i+1)*self.batch_size]
            batch_img_feats = [self.train_img_feats[n] for n in batch_img_ids]
            batch_txt_vecs = [self.train_txt_vecs[n] for n in batch_img_ids]
            batch_labels = [self.train_labels[n] for n in batch_img_ids]
            batch_labels_single = np.array([self.train_labels_single[n] for n in batch_img_ids])
            yield batch_img_feats, batch_txt_vecs, batch_labels, batch_labels_single, i

    def test_data(self):
        for i in range(self.num_test_batch):
            batch_img_ids = self.test_ids[i*self.batch_size : (i+1)*self.batch_size]
            batch_img_feats = [self.test_img_feats[n] for n in batch_img_ids]
            batch_txt_vecs = [self.test_txt_vecs[n] for n in batch_img_ids]
            batch_labels = [self.test_labels[n] for n in batch_img_ids]
            batch_labels_single = [self.test_labels_single[n] for n in batch_img_ids]
            yield batch_img_feats, batch_txt_vecs, batch_labels, batch_labels_single, i


class ModelParams(BaseModelParams):
    def __init__(self):
        BaseModelParams.__init__(self)

        self.batch_size = 64       
        self.visual_feat_dim = 4096
        #self.word_vec_dim = 200
        self.word_vec_dim = 1000
        self.lr_emb = 0.0001
        self.lr_domain = 0.0001
        self.top_k = 50
        self.semantic_emb_dim = 40
        self.dataset_name = 'nuswide'
        self.model_name = 'adv_semantic_zsl'
        self.model_dir = 'adv_semantic_zsl_%d_%d_%d' % (self.visual_feat_dim, self.word_vec_dim, self.semantic_emb_dim)
        self.checkpoint_dir = 'checkpoint'
        self.sample_dir = 'samples'
        self.dataset_dir = './data'
        self.log_dir = 'logs'

    def update(self):
        self.checkpoint_dir = os.path.join(self.checkpoint_dir, self.model_dir)
        self.sample_dir = os.path.join(self.sample_dir, self.model_dir)
        self.log_dir = os.path.join(self.log_dir, self.model_dir)
        self.dataset_dir = os.path.join(self.dataset_dir, self.dataset_name)


class AdvCrossModalSimple(BaseModel):
    def __init__(self, model_params,dataIter):
        BaseModel.__init__(self, model_params) 
        # self.data_iter = DataIter(self.model_params.batch_size)
        self.data_iter =dataIter
        self.visual_feats = tf.placeholder(tf.float32, [None, self.model_params.visual_feat_dim])
        self.word_vecs = tf.placeholder(tf.float32, [None, self.model_params.word_vec_dim])
        self.y = tf.placeholder(tf.int32, [self.model_params.batch_size,10])
        self.y_single = tf.placeholder(tf.int32, [self.model_params.batch_size,1])
        self.l = tf.placeholder(tf.float32, [])
        self.emb_v = self.visual_feature_embed(self.visual_feats)
        self.emb_w = self.label_embed(self.word_vecs)
        emb_v_ = tf.reduce_sum(self.emb_v, axis=1, keep_dims=True)
        emb_w_ = tf.reduce_sum(self.emb_w, axis=1, keep_dims=True)
        distance_map = tf.matmul(emb_v_,tf.ones([1,self.model_params.batch_size])) - tf.matmul(self.emb_v,tf.transpose(self.emb_w))+ \
            tf.matmul(tf.ones([self.model_params.batch_size,1]),tf.transpose(emb_w_))
        mask_initial = tf.to_float(tf.matmul(self.y_single,tf.ones([1,self.model_params.batch_size],dtype=tf.int32)) - \
            tf.matmul(tf.ones([self.model_params.batch_size,1],dtype=tf.int32),tf.transpose(self.y_single)))
        mask = tf.to_float(tf.not_equal(mask_initial, tf.zeros_like(mask_initial)))
        masked_dissimilar_loss = tf.multiply(distance_map,mask)
        self.dissimilar_loss = tf.reduce_mean(tf.maximum(0., 0.1*tf.ones_like(mask)-masked_dissimilar_loss))
        self.similar_loss = tf.sqrt(2 * tf.nn.l2_loss(self.emb_v - self.emb_w))
        self.similar_loss = tf.reduce_mean(self.similar_loss)
        logits_v = self.label_classifier(self.emb_v)
        logits_w = self.label_classifier(self.emb_w, reuse=True)
        self.label_loss = tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=logits_v) + \
            tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=logits_w)
        self.label_loss = tf.reduce_mean(self.label_loss)
        self.emb_loss = 50*self.label_loss + self.similar_loss + 0.2*self.dissimilar_loss
        self.emb_v_class = self.domain_classifier(self.emb_v, self.l)
        self.emb_w_class = self.domain_classifier(self.emb_w, self.l, reuse=True)

        all_emb_v = tf.concat([tf.ones([self.model_params.batch_size, 1]),
                                   tf.zeros([self.model_params.batch_size, 1])], 1)
        all_emb_w = tf.concat([tf.zeros([self.model_params.batch_size, 1]),
                                   tf.ones([self.model_params.batch_size, 1])], 1)
        self.domain_class_loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.emb_v_class, labels=all_emb_w) + \
            tf.nn.softmax_cross_entropy_with_logits(logits=self.emb_w_class, labels=all_emb_v)
        self.domain_class_loss = tf.reduce_mean(self.domain_class_loss)
        self.t_vars = tf.trainable_variables()
        self.vf_vars = [v for v in self.t_vars if 'vf_' in v.name]
        self.le_vars = [v for v in self.t_vars if 'le_' in v.name]
        self.dc_vars = [v for v in self.t_vars if 'dc_' in v.name]
        self.lc_vars = [v for v in self.t_vars if 'lc_' in v.name]
        

    def visual_feature_embed(self, X, is_training=True, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            net = tf.nn.tanh(slim.fully_connected(X, 4096, scope='vf_fc_0'))
            net = tf.nn.tanh(slim.fully_connected(net, 1000, scope='vf_fc_1'))
            net = tf.nn.tanh(slim.fully_connected(net, self.model_params.semantic_emb_dim, scope='vf_fc_2'))
        return net

    def label_embed(self, L, is_training=True, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            net = tf.nn.tanh(slim.fully_connected(L, 1000, scope='le_fc_0'))
            net = tf.nn.tanh(slim.fully_connected(net, 300, scope='le_fc_1'))
            net = tf.nn.tanh(slim.fully_connected(net, self.model_params.semantic_emb_dim, scope='le_fc_2'))
        return net 
    def label_classifier(self, X, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            net = slim.fully_connected(X, 10, scope='lc_fc_0')
        return net         
    def domain_classifier(self, E, l, is_training=True, reuse=False): 
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            E = flip_gradient(E, l)
            net = slim.fully_connected(E, self.model_params.semantic_emb_dim/2, scope='dc_fc_0')
            net = slim.fully_connected(net, self.model_params.semantic_emb_dim/4, scope='dc_fc_1')
            net = slim.fully_connected(net, 2, scope='dc_fc_2')
        return net

    def train_(self, sess):
        emb_train_op = tf.train.AdamOptimizer(
            learning_rate=self.model_params.lr_emb,
            beta1=0.5).minimize(self.emb_loss, var_list=self.le_vars+self.vf_vars)
        domain_train_op = tf.train.AdamOptimizer(
            learning_rate=self.model_params.lr_domain,
            beta1=0.5).minimize(self.domain_class_loss, var_list=self.dc_vars)
        tf.initialize_all_variables().run()
        self.saver = tf.train.Saver()
        self.model_params.epoch = 50
        start_time = time.time()
        losses=[]
        for epoch in range(self.model_params.epoch):         
            p = float(epoch) / self.model_params.epoch
            l = 2. / (1. + np.exp(-10. * p)) - 1
            for batch_feat, batch_vec, batch_labels, batch_labels_single, idx in self.data_iter.train_data():
                sess.run([emb_train_op, domain_train_op],
                         feed_dict={
                             self.visual_feats: batch_feat,
                             self.word_vecs: batch_vec,
                             self.y: batch_labels,
                             self.y_single: batch_labels_single[:,np.newaxis],
                             self.l: l})
              
                label_loss_val, similar_loss_val, emb_loss_val, domain_loss_val, dissimilar_loss_val = sess.run([self.label_loss, self.similar_loss, self.emb_loss, self.domain_class_loss, self.dissimilar_loss],
                                                         feed_dict={self.visual_feats: batch_feat,
                                                                    self.word_vecs: batch_vec,
                                                                    self.y: batch_labels,
                                                                    self.y_single: batch_labels_single[:,np.newaxis],
                                                                    self.l: l})
                print('Epoch: [%2d][%4d/%4d] time: %4.4f, emb_loss: %.8f, domain_loss: %.8f, label_loss: %.8f, similar_loss: %.8f, disimilar_loss: %.8f' %(
                    epoch, idx, self.data_iter.num_train_batch, time.time() - start_time, emb_loss_val, domain_loss_val, label_loss_val, similar_loss_val, dissimilar_loss_val
                ))
                sum_loss = emb_loss_val + domain_loss_val +label_loss_val +similar_loss_val +dissimilar_loss_val
                losses.append(sum_loss)
        params = {'emb_v':self.emb_v, 'dataIter': self.data_iter, 'emb_w': self.emb_w, 'visual_feats': self.visual_feats, 'word_vecs': self.word_vecs }
        logs = None
        losses=losses
        return params, losses, logs

    def eval(self, sess):
        test_img_feats_trans = []
        test_txt_vecs_trans = []
        test_labels = []
        for feats, vecs, _, labels, i in self.data_iter.test_data():
            feats_trans = sess.run(self.emb_v, feed_dict={self.visual_feats: feats})
            vecs_trans = sess.run(self.emb_w, feed_dict={self.word_vecs: vecs})
            #print("{0}".format(np.shape(labels)))            
            test_labels += list(labels)
            for ii in range(len(feats)):
                test_img_feats_trans.append(feats_trans[ii])
                test_txt_vecs_trans.append(vecs_trans[ii])
        test_img_feats_trans = np.asarray(test_img_feats_trans)
        test_txt_vecs_trans = np.asarray(test_txt_vecs_trans)

        avg_precs = []
        all_precs = []
        all_k = [50]
        for k in all_k: 
            for i in range(len(test_txt_vecs_trans)):
                query_label = test_labels[i]

                # distances and sort by distances
                wv = test_txt_vecs_trans[i]
                diffs = test_img_feats_trans - wv
                dists = np.linalg.norm(diffs, axis=1)
                sorted_idx = np.argsort(dists)
                precs = []
                for topk in range(1, k + 1):
                    hits = 0
                    top_k = sorted_idx[0 : topk]                    
                    if query_label != test_labels[top_k[-1]]:
                        continue
                    for ii in top_k:
                        retrieved_label = test_labels[ii]
                        if retrieved_label == query_label:
                            hits += 1
                    precs.append(float(hits) / float(topk))
                if len(precs) == 0:
                    precs.append(0)
                avg_precs.append(np.average(precs))
            mean_avg_prec = np.mean(avg_precs)
            all_precs.append(mean_avg_prec)
        print('[Eval - txt2img] mAP: %f ' % (all_precs[0]))

        avg_precs = []
        all_precs = []
        all_k = [50]
        for k in all_k:        
            for i in range(len(test_img_feats_trans)):
                query_img_feat = test_img_feats_trans[i]
                ground_truth_label = test_labels[i]

                # calculate distance and sort
                diffs = test_txt_vecs_trans - query_img_feat
                dists = np.linalg.norm(diffs, axis=1)
                sorted_idx = np.argsort(dists)
                precs = []
                for topk in range(1, k + 1):
                    hits = 0
                    top_k = sorted_idx[0 : topk]
                    if ground_truth_label != test_labels[top_k[-1]]:
                        continue
                    for ii in top_k:
                        retrieved_label = test_labels[ii]
                        if ground_truth_label == retrieved_label:
                            hits += 1
                    precs.append(float(hits) / float(topk))
                if len(precs) == 0:
                    precs.append(0)
                avg_precs.append(np.average(precs))
            mean_avg_prec = np.mean(avg_precs)
            all_precs.append(mean_avg_prec)           
        print('[Eval - img2txt] mAP: %f ' % (all_precs[0]))


W0520 07:18:42.093694 140682794006400 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [15]:
def predict_(dataset_obj, params, tag,sess):  
  if tag =='test':
    data_iter = params['dataIter']
    emb_v = params['emb_v']
    emb_w = params['emb_w']
    # sess = params['sess']
    visual_feats = params['visual_feats']
    word_vecs = params['word_vecs']
    test_img_feats_trans = []
    test_txt_vecs_trans = []
    test_labels = []
    for feats, vecs, labels, i in data_iter.test_data():
        feats_trans = sess.run(emb_v, feed_dict={visual_feats: feats})
        vecs_trans = sess.run(emb_w, feed_dict={word_vecs: vecs})
        #print("{0}".format(np.shape(labels)))            
        test_labels += list(labels)
        for ii in range(len(feats)):
            test_img_feats_trans.append(feats_trans[ii])
            test_txt_vecs_trans.append(vecs_trans[ii])
    test_img_feats_trans = np.asarray(test_img_feats_trans)
    test_txt_vecs_trans = np.asarray(test_txt_vecs_trans)

    retrieval_t_i = []
    number_of_queries = len(test_txt_vecs_trans)
    for i in range(len(test_txt_vecs_trans)):
        query_label = test_labels[i]
        # distances and sort by distances
        wv = test_txt_vecs_trans[i]
        diffs = test_img_feats_trans - wv
        dists = np.linalg.norm(diffs, axis=1)
        sorted_idx = np.argsort(dists)
        retrieval_t_i.append(sorted_idx)

    retrieval_i_t = []
    for i in range(len(test_img_feats_trans)):
        query_img_feat = test_img_feats_trans[i]
        ground_truth_label = test_labels[i]

        # calculate distance and sort
        diffs = test_txt_vecs_trans - query_img_feat
        dists = np.linalg.norm(diffs, axis=1)
        sorted_idx = np.argsort(dists)
        retrieval_i_t.append(sorted_idx)
    retrieval = {'itot_ranked_results':np.array(retrieval_i_t), 'ttoi_ranked_results':np.array(retrieval_t_i), 'number_of_queries': number_of_queries }
  return number_of_queries, retrieval, None

# Wiki


In [16]:
def dummy_predict(dataset_obj, params, tag):
  n_samples = params['number_of_queries']
  results = {'itot_ranked_results':params['itot_ranked_results'].copy(), 'ttoi_ranked_results':params['ttoi_ranked_results'].copy()}
  return n_samples, results, None


In [22]:
from __future__ import print_function
import os, time, cPickle
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from random import shuffle
import sklearn.preprocessing
# from base_model import BaseModel, BaseModelParams, BaseDataIter
#import utils
# from flip_gradient import flip_gradient

class DataIter(BaseDataIter):
    def __init__(self, batch_size,data):
        BaseDataIter.__init__(self, batch_size)
        self.num_train_batch = 0
        self.num_test_batch = 0

        self.train_img_feats=data.x_train['img_train']
        self.train_txt_vecs=data.x_train['txt_train']
        self.train_labels=data.y_train['train_labels']
        self.test_img_feats=data.x_test['img_test']
        self.test_txt_vecs=data.x_test['txt_test']
        self.test_labels=data.y_test['test_labels']   
        self.num_train_batch = len(self.train_img_feats) / self.batch_size
        self.num_test_batch = len(self.test_img_feats) / self.batch_size

    def train_data(self):
        for i in range(self.num_train_batch):
            batch_img_feats = self.train_img_feats[i*self.batch_size : (i+1)*self.batch_size]
            batch_txt_vecs = self.train_txt_vecs[i*self.batch_size : (i+1)*self.batch_size]
            batch_labels = self.train_labels[i*self.batch_size : (i+1)*self.batch_size]
            yield batch_img_feats, batch_txt_vecs, batch_labels, i

    def test_data(self):
        for i in range(self.num_test_batch):
            batch_img_feats = self.test_img_feats[i*self.batch_size : (i+1)*self.batch_size]
            batch_txt_vecs = self.test_txt_vecs[i*self.batch_size : (i+1)*self.batch_size]
            batch_labels = self.test_labels[i*self.batch_size : (i+1)*self.batch_size]
            yield batch_img_feats, batch_txt_vecs, batch_labels, i


class ModelParams(BaseModelParams):
    def __init__(self):
        BaseModelParams.__init__(self)

        self.epoch = 200
        self.batch_size = 64
        self.visual_feat_dim = 4096
        #self.word_vec_dim = 300
        self.word_vec_dim = 5000
        self.lr_total = 0.0001
        self.lr_emb = 0.0001
        self.lr_domain = 0.0001
        self.top_k = 50
        self.semantic_emb_dim = 40
        self.dataset_name = 'wikipedia_dataset'
        self.model_name = 'adv_semantic_zsl'
        self.model_dir = 'adv_semantic_zsl_%d_%d_%d' % (self.visual_feat_dim, self.word_vec_dim, self.semantic_emb_dim)

        self.checkpoint_dir = 'checkpoint'
        self.sample_dir = 'samples'
        self.dataset_dir = './data'
        self.log_dir = 'logs'
 
    def update(self):
        self.checkpoint_dir = os.path.join(self.checkpoint_dir, self.model_dir)
        self.sample_dir = os.path.join(self.sample_dir, self.model_dir)
        self.log_dir = os.path.join(self.log_dir, self.model_dir)
        self.dataset_dir = os.path.join(self.dataset_dir, self.dataset_name)


class AdvCrossModalSimple(BaseModel):
    def __init__(self, model_params,dataIter):
        BaseModel.__init__(self, model_params)
        # self.data_iter = DataIter(self.model_params.batch_size)
        self.data_iter = dataIter
        self.visual_feats = tf.placeholder(tf.float32, [None, self.model_params.visual_feat_dim])
        self.word_vecs = tf.placeholder(tf.float32, [None, self.model_params.word_vec_dim])
        self.y = tf.placeholder(tf.int32, [self.model_params.batch_size,10])
        self.y_single = tf.placeholder(tf.int32, [self.model_params.batch_size,1])
        self.l = tf.placeholder(tf.float32, [])
        self.emb_v = self.visual_feature_embed(self.visual_feats)
        self.emb_w = self.label_embed(self.word_vecs)
        #self.corr_loss = tf.sqrt(2 * tf.nn.l2_loss(self.emb_v - self.emb_w))
        #self.corr_loss = tf.reduce_mean(self.corr_loss)

        # dissimilar loss
        emb_v_ = tf.reduce_sum(self.emb_v, axis=1, keep_dims=True)
        emb_w_ = tf.reduce_sum(self.emb_w, axis=1, keep_dims=True)
        distance_map = tf.matmul(emb_v_,tf.ones([1,self.model_params.batch_size])) - tf.matmul(self.emb_v,tf.transpose(self.emb_w))+ \
            tf.matmul(tf.ones([self.model_params.batch_size,1]),tf.transpose(emb_w_))
        mask_initial = tf.to_float(tf.matmul(self.y_single,tf.ones([1,self.model_params.batch_size],dtype=tf.int32)) - \
            tf.matmul(tf.ones([self.model_params.batch_size,1],dtype=tf.int32),tf.transpose(self.y_single)))
        mask = tf.to_float(tf.not_equal(mask_initial, tf.zeros_like(mask_initial)))
        masked_dissimilar_loss = tf.multiply(distance_map,mask)
        self.dissimilar_loss = tf.reduce_mean(tf.maximum(0., 0.1*tf.ones_like(mask)-masked_dissimilar_loss))
        #self.similar_loss = tf.reduce_mean(tf.abs(distance_map-masked_dissimilar_loss))
        self.similar_loss = tf.sqrt(2 * tf.nn.l2_loss(self.emb_v - self.emb_w))
        self.similar_loss = tf.reduce_mean(self.similar_loss)
        logits_v = self.label_classifier(self.emb_v)
        logits_w = self.label_classifier(self.emb_w, reuse=True)
        self.label_loss = tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=logits_v) + \
            tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=logits_w)
        self.label_loss = tf.reduce_mean(self.label_loss)
        self.emb_loss = 100*self.label_loss + self.similar_loss + 0.02*self.dissimilar_loss
        self.emb_v_class = self.domain_classifier(self.emb_v, self.l)
        self.emb_w_class = self.domain_classifier(self.emb_w, self.l, reuse=True)

        all_emb_v = tf.concat([tf.ones([self.model_params.batch_size, 1]),
                                   tf.zeros([self.model_params.batch_size, 1])], 1)
        all_emb_w = tf.concat([tf.zeros([self.model_params.batch_size, 1]),
                                   tf.ones([self.model_params.batch_size, 1])], 1)
        self.domain_class_loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.emb_v_class, labels=all_emb_w) + \
            tf.nn.softmax_cross_entropy_with_logits(logits=self.emb_w_class, labels=all_emb_v)
        self.domain_class_loss = tf.reduce_mean(self.domain_class_loss)

        self.t_vars = tf.trainable_variables()
        self.vf_vars = [v for v in self.t_vars if 'vf_' in v.name]
        self.le_vars = [v for v in self.t_vars if 'le_' in v.name]
        self.dc_vars = [v for v in self.t_vars if 'dc_' in v.name]
        self.lc_vars = [v for v in self.t_vars if 'lc_' in v.name]
    def visual_feature_embed(self, X, is_training=True, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            net = tf.nn.tanh(slim.fully_connected(X, 512, scope='vf_fc_0'))
            net = tf.nn.tanh(slim.fully_connected(net, 100, scope='vf_fc_1'))
            net = tf.nn.tanh(slim.fully_connected(net, self.model_params.semantic_emb_dim, scope='vf_fc_2'))
        return net

    def label_embed(self, L, is_training=True, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            net = tf.nn.tanh(slim.fully_connected(L, self.model_params.semantic_emb_dim, scope='le_fc_0'))
            net = tf.nn.tanh(slim.fully_connected(net, 100, scope='le_fc_1'))
            net = tf.nn.tanh(slim.fully_connected(net, self.model_params.semantic_emb_dim, scope='le_fc_2'))
        return net 
    def label_classifier(self, X, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            net = slim.fully_connected(X, 10, scope='lc_fc_0')
        return net         
    def domain_classifier(self, E, l, is_training=True, reuse=False):
        with slim.arg_scope([slim.fully_connected], activation_fn=None, reuse=reuse):
            E = flip_gradient(E, l)
            net = slim.fully_connected(E, self.model_params.semantic_emb_dim/2, scope='dc_fc_0')
            net = slim.fully_connected(net, self.model_params.semantic_emb_dim/4, scope='dc_fc_1')
            net = slim.fully_connected(net, 2, scope='dc_fc_2')
        return net

    def train(self, sess):
        #self.check_dirs()
 
        total_loss = self.emb_loss + self.domain_class_loss
        total_train_op = tf.train.AdamOptimizer(
            learning_rate=self.model_params.lr_total,
            beta1=0.5).minimize(total_loss)
        emb_train_op = tf.train.AdamOptimizer(
            learning_rate=self.model_params.lr_emb,
            beta1=0.5).minimize(self.emb_loss, var_list=self.le_vars+self.vf_vars)
        domain_train_op = tf.train.AdamOptimizer(
            learning_rate=self.model_params.lr_domain,
            beta1=0.5).minimize(self.domain_class_loss, var_list=self.dc_vars)

        tf.initialize_all_variables().run()
        self.saver = tf.train.Saver()

        start_time = time.time()
        map_avg_ti = []
        map_avg_it = []
        adv_loss = []
        emb_loss = []
        losses=[]
        for epoch in range(self.model_params.epoch):
            if epoch % 1 ==0:
                adv = 0
                emb = 0 
                test_img_feats_trans = []
                test_txt_vecs_trans = []
                test_labels = []
                p = float(epoch) / self.model_params.epoch
                l = 2. / (1. + np.exp(-10. * p)) - 1 
                for feats, vecs, labels, i in self.data_iter.train_data():
                    labels_ = labels - np.ones_like(labels)
                    label_binarizer = sklearn.preprocessing.LabelBinarizer()
                    label_binarizer.fit(range(max(labels_)+1))
                    b = label_binarizer.transform(labels_)                     
                    emb_loss_val, domain_loss_val = sess.run([self.emb_loss, self.domain_class_loss],
                                                         feed_dict={self.visual_feats: feats,
                                                                    self.word_vecs: vecs,
                                                                    self.y: b,
                                                                    self.y_single: np.transpose([labels]),
                                                                    self.l: l})
                    emb += emb_loss_val
                    adv += domain_loss_val
                emb = float(emb)/float(25)
                adv = float(adv)/float(25)
                adv_loss.append(adv)
                emb_loss.append(emb)                                              
              
                for feats, vecs, labels, i in self.data_iter.test_data():           
                    feats_trans = sess.run(self.emb_v, feed_dict={self.visual_feats: feats})
                    vecs_trans = sess.run(self.emb_w, feed_dict={self.word_vecs: vecs})
                    test_labels += labels
                    for ii in range(len(feats)):
                        test_img_feats_trans.append(feats_trans[ii])
                        test_txt_vecs_trans.append(vecs_trans[ii])
                test_img_feats_trans = np.asarray(test_img_feats_trans)
                test_txt_vecs_trans = np.asarray(test_txt_vecs_trans)

                k = self.model_params.top_k
                avg_precs = []
                for i in range(len(test_txt_vecs_trans)):
                    query_label = test_labels[i]
                    # distances and sort by distances
                    wv = test_txt_vecs_trans[i]
                    diffs = test_img_feats_trans - wv
                    dists = np.linalg.norm(diffs, axis=1)
                    sorted_idx = np.argsort(dists)
                    # for each k do top-k
                    precs = []
                    for topk in range(1, k + 1):
                        hits = 0
                        top_k = sorted_idx[0 : topk]
                        if query_label != test_labels[top_k[-1]]:
                            continue
                        for ii in top_k:
                            retrieved_label = test_labels[ii]
                            if retrieved_label == query_label:
                                hits += 1
                        precs.append(float(hits) / float(topk))
                    if len(precs) == 0:
                        precs.append(0)
                    avg_precs.append(np.average(precs))
                map_avg_ti.append(np.mean(avg_precs))
                
                avg_precs_ = []
                for i in range(len(test_img_feats_trans)):
                    query_img_feat = test_img_feats_trans[i]
                    ground_truth_label = test_labels[i]
                    # calculate distance and sort
                    diffs = test_txt_vecs_trans - query_img_feat
                    dists = np.linalg.norm(diffs, axis=1)
                    sorted_idx = np.argsort(dists)
                    # for each k in top-k
                    precs = []
                    for topk in range(1, k + 1):
                        hits = 0
                        top_k = sorted_idx[0 : topk]
                        if ground_truth_label != test_labels[top_k[-1]]:
                            continue
                        for ii in top_k:
                            retrieved_label = test_labels[ii]
                            if ground_truth_label == retrieved_label:
                                hits += 1
                        precs.append(float(hits) / float(topk))
                    if len(precs) == 0:
                        precs.append(0)
                    avg_precs_.append(np.average(precs))
                map_avg_it.append(np.mean(avg_precs_)) 
            if epoch == (self.model_params.epoch-1):
                with open('./data/wikipedia_dataset/map_avg_ti.pkl', 'wb') as f:
                    cPickle.dump(map_avg_ti, f, cPickle.HIGHEST_PROTOCOL)
                with open('./data/wikipedia_dataset/map_avg_it.pkl', 'wb') as f:
                    cPickle.dump(map_avg_it, f, cPickle.HIGHEST_PROTOCOL)       
                with open('./data/wikipedia_dataset/adv_loss.pkl', 'wb') as f:
                    cPickle.dump(adv_loss, f, cPickle.HIGHEST_PROTOCOL)
                with open('./data/wikipedia_dataset/emb_loss.pkl', 'wb') as f:
                    cPickle.dump(emb_loss, f, cPickle.HIGHEST_PROTOCOL)                                    
                print('dump map_avg')
                                                   
            if epoch==0:
                test_img_feats_trans = []
                test_txt_vecs_trans = []                          
                for feats, vecs, labels, i in self.data_iter.train_data():
                    feats_trans = sess.run(self.emb_v, feed_dict={self.visual_feats: feats})
                    vecs_trans = sess.run(self.emb_w, feed_dict={self.word_vecs: vecs})
                    for ii in range(len(feats)):
                        test_img_feats_trans.append(feats_trans[ii])
                        test_txt_vecs_trans.append(vecs_trans[ii])
                test_img_feats_trans = np.asarray(test_img_feats_trans)
                test_txt_vecs_trans = np.asarray(test_txt_vecs_trans)
                test_feats_trans = np.concatenate((test_img_feats_trans[0:1000], test_txt_vecs_trans[-1000:]))
                with open('./data/wikipedia_dataset/before_feats_transformed.pkl', 'wb') as f:
                    cPickle.dump(test_feats_trans, f, cPickle.HIGHEST_PROTOCOL)               
                print('dump before')   
                        
            p = float(epoch) / self.model_params.epoch
            l = 2. / (1. + np.exp(-10. * p)) - 1
            for batch_feat, batch_vec, batch_labels, idx in self.data_iter.train_data():
                # create one-hot labels
                batch_labels_ = batch_labels - np.ones_like(batch_labels)
                label_binarizer = sklearn.preprocessing.LabelBinarizer()
                label_binarizer.fit(range(max(batch_labels_)+1))
                b = label_binarizer.transform(batch_labels_)
                #_, label_loss_val, dissimilar_loss_val, similar_loss_val = sess.run([total_train_op, self.label_loss, self.dissimilar_loss, self.similar_loss], feed_dict={self.visual_feats: batch_feat, self.word_vecs: batch_vec, self.y: b, self.y_single: np.transpose([batch_labels]),self.l: l})
                sess.run([emb_train_op, domain_train_op],
                         feed_dict={
                             self.visual_feats: batch_feat,
                             self.word_vecs: batch_vec,
                             self.y: b,
                             self.y_single: np.transpose([batch_labels_]),
                             self.l: l})
                label_loss_val, similar_loss_val, emb_loss_val, domain_loss_val, dissimilar_loss_val= sess.run([self.label_loss, self.similar_loss, self.emb_loss, self.domain_class_loss, self.dissimilar_loss],
                                                         feed_dict={self.visual_feats: batch_feat,
                                                                    self.word_vecs: batch_vec,
                                                                    self.y: b,
                                                                    self.y_single: np.transpose([batch_labels]),
                                                                    self.l: l})
                print('Epoch: [%2d][%4d/%4d] time: %4.4f, emb_loss: %.8f, domain_loss: %.8f, label_loss: %.8f, similar_loss: %.8f, dissimilar_loss: %.8f' %(
                    epoch, idx, self.data_iter.num_train_batch, time.time() - start_time, emb_loss_val, domain_loss_val, label_loss_val, similar_loss_val, dissimilar_loss_val
                ))
                sum_loss = emb_loss_val + domain_loss_val +label_loss_val +similar_loss_val +dissimilar_loss_val
                losses.append(sum_loss)
        params = {'emb_v':self.emb_v, 'dataIter': self.data_iter, 'emb_w': self.emb_w, 'visual_feats': self.visual_feats, 'word_vecs': self.word_vecs }
        logs = None
        losses=losses
        return params, losses, logs
                      

    # def eval(self, sess):
    #     start = time.time()
    #     test_img_feats_trans = []
    #     test_txt_vecs_trans = []
    #     test_labels = []
    #     for feats, vecs, labels, i in self.data_iter.test_data():
    #         feats_trans = sess.run(self.emb_v, feed_dict={self.visual_feats: feats})
    #         vecs_trans = sess.run(self.emb_w, feed_dict={self.word_vecs: vecs})
    #         test_labels += labels      
    #         for ii in range(len(feats)):
    #             test_img_feats_trans.append(feats_trans[ii])
    #             test_txt_vecs_trans.append(vecs_trans[ii])
    #     test_img_feats_trans = np.asarray(test_img_feats_trans)
    #     test_txt_vecs_trans = np.asarray(test_txt_vecs_trans)
    #     test_feats_trans = np.concatenate((test_img_feats_trans[0:1000], test_txt_vecs_trans[-1000:]))
    #     #with open('./data/wikipedia_dataset/test_feats_transformed.pkl', 'wb') as f:
    #     #    cPickle.dump(test_feats_trans, f, cPickle.HIGHEST_PROTOCOL)        
    #     with open('./data/wiki_shallow/test_feats_transformed.pkl', 'wb') as f:
    #         cPickle.dump(test_feats_trans, f, cPickle.HIGHEST_PROTOCOL)                   
    #     print('[Eval] transformed test features in %4.4f' % (time.time() - start))
        # #k = self.model_params.top_k
        # avg_precs = []
        # all_precs = []
        # #all_k = [5,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]
        # all_k =[50]
        # for k in all_k:
        #     for i in range(len(test_txt_vecs_trans)):
        #         query_label = test_labels[i]

        #         # distances and sort by distances
        #         wv = test_txt_vecs_trans[i]
        #         diffs = test_img_feats_trans - wv
        #         dists = np.linalg.norm(diffs, axis=1)
        #         sorted_idx = np.argsort(dists)

        #         #for each k do top-k
        #         precs = []
        #         for topk in range(1, k + 1):
        #             hits = 0
        #             top_k = sorted_idx[0 : topk]
        #             if np.sum(query_label) != test_labels[top_k[-1]]:
        #                 continue
        #             for ii in top_k:
        #                 retrieved_label = test_labels[ii]
        #                 if np.sum(retrieved_label) == query_label:
        #                     hits += 1
        #             precs.append(float(hits) / float(topk))
        #         if len(precs) == 0:
        #             precs.append(0)
        #         avg_precs.append(np.average(precs))
        #     mean_avg_prec = np.mean(avg_precs)
        #     all_precs.append(mean_avg_prec)
        # print('[Eval - txt2img] mAP: %f in %4.4fs' % (all_precs[0], (time.time() - start)))
        # t2i = all_precs[0]
        # #with open('./data/wikipedia_dataset/txt2img_all_precision.pkl', 'wb') as f:
        # #    cPickle.dump(all_precs, f, cPickle.HIGHEST_PROTOCOL)     
        # with open('./data/wiki_shallow/txt2img_all_precision.pkl', 'wb') as f:
        #     cPickle.dump(all_precs, f, cPickle.HIGHEST_PROTOCOL)                  

        # avg_precs = []
        # all_precs = []
        # #all_k = [5,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]
        # all_k = [50]
        # for k in all_k:        
        #     for i in range(len(test_img_feats_trans)):
        #         query_img_feat = test_img_feats_trans[i]
        #         ground_truth_label = test_labels[i]

        #         # calculate distance and sort
        #         diffs = test_txt_vecs_trans - query_img_feat
        #         dists = np.linalg.norm(diffs, axis=1)
        #         sorted_idx = np.argsort(dists)

        #         # for each k in top-k
        #         precs = []
        #         for topk in range(1, k + 1):
        #             hits = 0
        #             top_k = sorted_idx[0 : topk]
        #             if np.sum(ground_truth_label) != test_labels[top_k[-1]]:
        #                 continue
        #             for ii in top_k:
        #                 retrieved_label = test_labels[ii]
        #                 if np.sum(ground_truth_label) == retrieved_label:
        #                     hits += 1
        #             precs.append(float(hits) / float(topk))
        #         if len(precs) == 0:
        #             precs.append(0)
        #         avg_precs.append(np.average(precs))
        #     mean_avg_prec = np.mean(avg_precs)
        #     all_precs.append(mean_avg_prec)            
        # print('[Eval - img2txt] mAP: %f in %4.4fs' % (all_precs[0], (time.time() - start)))

In [18]:
import pickle
import numpy as np

def data_loader(dirpath, tag):
  if tag=='train':
    with open('./data/wikipedia_dataset/train_img_feats.pkl', 'rb') as f:
        train_img_feats = cPickle.load(f)
    with open('./data/wikipedia_dataset/train_txt_vecs.pkl', 'rb') as f:
        train_txt_vecs = cPickle.load(f)
    with open('./data/wikipedia_dataset/train_labels.pkl', 'rb') as f:
        train_labels = cPickle.load(f)
    
    return {'img_train': train_img_feats, 'txt_train': train_txt_vecs}, {'train_labels': train_labels}
    
  if tag =='test':
    with open('./data/wikipedia_dataset/test_img_feats.pkl', 'rb') as f:
        test_img_feats = cPickle.load(f)
    with open('./data/wikipedia_dataset/test_txt_vecs.pkl', 'rb') as f:
        test_txt_vecs = cPickle.load(f)
    with open('./data/wikipedia_dataset/test_labels.pkl', 'rb') as f:
        test_labels = cPickle.load(f)
    return {'img_test': test_img_feats, 'txt_test': test_txt_vecs}, {'test_labels': test_labels}


In [19]:
def predict_(dataset_obj, params, tag, sess):
    if tag =='test':
        data_iter = params['dataIter']
        emb_v = params['emb_v']
        emb_w = params['emb_w']
        # sess = params['sess']
        visual_feats = params['visual_feats']
        word_vecs = params['word_vecs']
        test_img_feats_trans = []
        test_txt_vecs_trans = []
        test_labels = []
        for feats, vecs, labels, i in data_iter.test_data():
            feats_trans = sess.run(emb_v, feed_dict={visual_feats: feats})
            vecs_trans = sess.run(emb_w, feed_dict={word_vecs: vecs})
            #print("{0}".format(np.shape(labels)))            
            test_labels += list(labels)
            for ii in range(len(feats)):
                test_img_feats_trans.append(feats_trans[ii])
                test_txt_vecs_trans.append(vecs_trans[ii])
        test_img_feats_trans = np.asarray(test_img_feats_trans)
        test_txt_vecs_trans = np.asarray(test_txt_vecs_trans)

    retrieval_t_i = []
    number_of_queries = len(test_txt_vecs_trans)
    for i in range(len(test_txt_vecs_trans)):
        query_label = test_labels[i]
        # distances and sort by distances
        wv = test_txt_vecs_trans[i]
        diffs = test_img_feats_trans - wv
        dists = np.linalg.norm(diffs, axis=1)
        sorted_idx = np.argsort(dists)
        retrieval_t_i.append(sorted_idx)

    retrieval_i_t = []
    for i in range(len(test_img_feats_trans)):
        query_img_feat = test_img_feats_trans[i]
        ground_truth_label = test_labels[i]

        # calculate distance and sort
        diffs = test_txt_vecs_trans - query_img_feat
        dists = np.linalg.norm(diffs, axis=1)
        sorted_idx = np.argsort(dists)
        retrieval_i_t.append(sorted_idx)
    retrieval = {'itot_ranked_results':np.array(retrieval_i_t), 'ttoi_ranked_results':np.array(retrieval_t_i), 'number_of_queries': number_of_queries }
    return number_of_queries, retrieval, None


In [23]:
import tensorflow as tf
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
# from models.adv_crossmodal_simple_nuswide import AdvCrossModalSimple, ModelParams, DataIter
# from framework.dataset import Dataset
# from framework.model import Parameters, Model
def train(dataset_obj, parameters, hyperparams):
    
    print("Inside Model -- train ( basically main of ACMR)")
    graph = tf.Graph()
    model_params = hyperparams
    model_params.update()
    with graph.as_default():     
        dataIter = DataIter(hyperparams.batch_size,dataset_obj)   
        model = AdvCrossModalSimple(model_params,dataIter)
    with tf.Session(graph=graph) as sess:
      parameters_, losses, logs = model.train(sess)      
      # model.eval(sess)
      number_of_queries, retrieval, logs =predict_(dataset_obj, parameters_, 'test',sess)
      return retrieval, losses, logs

def main(_):  
    nuswide_filepath_train = "./data/wikipedia_dataset/"
    nuswide_filepath_valid ="."
    nuswide_filepath_test="./data/wikipedia_dataset/"
    data = Dataset((nuswide_filepath_train, nuswide_filepath_valid, nuswide_filepath_test), data_loader,
                   read_directories=(True, False, True))
    data.load_data()
    print("Data loaded successfully")
    hyperparams = ModelParams()
    model = Model(train,hyperparams,data,prediction_function=dummy_predict)
    model.train_model()
    model.predict('test')
    label_=data.y_test['test_labels']  
    mat_label=[]  
    for key in label_:
      mat_label.append(label_[key])
    y_mat_label = np.array(mat_label)
    data = y_mat_label[0:640]
    encoded = to_categorical(data)
    inverted = argmax(encoded[0])
    model.evaluate(encoded,encoded)
    model.save_stats("acmr_wikipedia_stats.npy")

if __name__ == '__main__':
    tf.app.run()

Streaming output truncated to the last 5000 lines.
Epoch: [48][  24/  33] time: 36.1685, emb_loss: 15.14096355, domain_loss: 1.38492346, label_loss: 0.09400660, similar_loss: 5.71518278, dissimilar_loss: 1.25603044
Epoch: [48][  25/  33] time: 36.1780, emb_loss: 15.13676167, domain_loss: 1.38124311, label_loss: 0.09300053, similar_loss: 5.80649328, dissimilar_loss: 1.51075649
Epoch: [48][  26/  33] time: 36.1875, emb_loss: 14.03639126, domain_loss: 1.39620900, label_loss: 0.08939348, similar_loss: 5.07576847, dissimilar_loss: 1.06370938
Epoch: [48][  27/  33] time: 36.1973, emb_loss: 15.00292683, domain_loss: 1.39787996, label_loss: 0.10042869, similar_loss: 4.92868185, dissimilar_loss: 1.56875670
Epoch: [48][  28/  33] time: 36.2087, emb_loss: 16.60484123, domain_loss: 1.39882016, label_loss: 0.08746498, similar_loss: 7.83140945, dissimilar_loss: 1.34671569
Epoch: [48][  29/  33] time: 36.2189, emb_loss: 20.65407372, domain_loss: 1.40133452, label_loss: 0.10878368, similar_loss: 9.754

SystemExit: ignored